# MNIST for Experts

### CNNを使って精度改善を狙う

In [46]:
from tensorflow.examples.tutorials.mnist import input_data

In [47]:
# one_hot:1つだけ1で，それ以外は0
mnist = input_data.read_data_sets('MNIST_data', one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [48]:
import tensorflow as tf
# C++ backendへの接続とコード構成を柔軟化する
sess = tf.InteractiveSession()

In [60]:
# 入力画像と目的出力クラスのためのノードを作成することで計算グラフの組み立てを開始
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [61]:
# variables(784 inputs, 10 classes)
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

### 多層畳み込みネットワークの構築

In [62]:
# 重みの初期化
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

In [63]:
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [64]:
# 畳み込み
def conv2d(x, W):
    # strides: 1 pixel ずらして畳み込み, padding:0なので出力==入力('SAME')
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

In [65]:
# プーリング:サイズを小さくして特徴量を抽出
# MAX pooling:2×2ブロック内の最大値を採用して数値のゆらぎを削減, strides:2pixelずらす
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [66]:
# 第一畳み込み層(5 * 5のマスに1つの1つの値，それが32個)
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

In [67]:
# -1:式の結果に合わせて動的にパラメータを決定
x_image = tf.reshape(x, [-1, 28, 28, 1])
print(x_image)

Tensor("Reshape_14:0", shape=(?, 28, 28, 1), dtype=float32)


In [72]:
# Filtered valueをReLuで値を変換, 特徴のない値を切り捨てて計算量を削減
# 1層目にフィルター
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

In [73]:
# 次元圧縮
h_pool1 = max_pool_2x2(h_conv1)

In [74]:
# 第二層
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2) # 次元削減

In [75]:
# 
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [76]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [77]:
# 読み出し層
W_fc2 = weight_variable([1024, 10]) # 1024行 × 0 ~ 9
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [78]:
# 各
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_conv, labels = y_)) # https://github.com/ibab/tensorflow-wavenet/issues/223
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy) # grad descent

# 正解数: y_conv最大値を返す
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))

# 精度
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# モデルの訓練
sess.run(tf.global_variables_initializer())
for i in range(2000): # 20000
    # 次のランダムな50個を取り出す
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict = {x:batch[0], y_:batch[1],
                                                    keep_prob:1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x:batch[0], y_:batch[1], keep_prob:0.5})
print("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images,
                                                 y_:mnist.test.labels,
                                                 keep_prob: 1.0}))       

step 0, training accuracy 0.1
step 100, training accuracy 0.78
step 200, training accuracy 0.84
step 300, training accuracy 0.94
step 400, training accuracy 0.92
step 500, training accuracy 1
step 600, training accuracy 0.92
step 700, training accuracy 0.94
step 800, training accuracy 0.96
step 900, training accuracy 0.96
step 1000, training accuracy 0.98
step 1100, training accuracy 0.94
step 1200, training accuracy 0.98
step 1300, training accuracy 0.94
step 1400, training accuracy 0.94
step 1500, training accuracy 0.96
step 1600, training accuracy 0.96
step 1700, training accuracy 1
step 1800, training accuracy 0.98
step 1900, training accuracy 0.98
test accuracy 0.9768
